In [1]:
from matplotlib import pyplot as plt
from matplotlib import ticker
import datetime
import os
import numpy as np
import sys
sys.path.insert(0, '../')


SMALL_SIZE = 20
MEDIUM_SIZE = 25
BIGGER_SIZE = 30

plt.rc('font', family='Arial', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
plt.rc('grid', linestyle='--')
plt.rc('grid', color='#E68F6B')
from ast import literal_eval as make_tuple
def get_param_from_file(file_name, name):
    range_ = ""
    with open(file_name, "r") as f:
        for line in f.readlines():
            if name in line:
                range_ = line
    return make_tuple(range_.strip().split("=")[-1])

def get_scan_range(file_name):
    return get_param_from_file(file_name, "scan range")

def get_confocal_scan_range(file_name):
    return  get_param_from_file(file_name, "x scan range"),  get_param_from_file(file_name, "y scan range")


import numpy as np
import matplotlib.pyplot as plt
from lmfit import Model

# define the Lorentzian function
def multiple_lorentzian(x, centers, sigmas, amplitudes):
    """ Mathematical definition of the sum of multiple (physical) Lorentzian functions without any
    bias.

    WARNING: iterable parameters "centers", "sigmas" and "amplitudes" must have same length.

    @param float x: The independent variable to calculate lorentz(x)
    @param iterable centers: Iterable containing center positions for all lorentzians
    @param iterable sigmas: Iterable containing sigmas for all lorentzians
    @param iterable amplitudes: Iterable containing amplitudes for all lorentzians
    """
    assert len(centers) == len(sigmas) == len(amplitudes)
    return sum(amp * sig ** 2 / ((x - c) ** 2 + sig ** 2) for c, sig, amp in
               zip(centers, sigmas, amplitudes))

def lorentzian(x, offset, center, sigma, amplitude):
    return offset + multiple_lorentzian(x, (center,), (sigma,), (amplitude,))

def double_lorentzian(x, offset, center_1, center_2, sigma_1, sigma_2, amplitude_1, amplitude_2):
    return offset + multiple_lorentzian(x,
                                        (center_1, center_2),
                                        (sigma_1, sigma_2),
                                        (amplitude_1, amplitude_2))

def multi_lorentzians(x, *params):
    num_lorentzians = len(params) // 3
    result = np.zeros_like(x)
    for i in range(num_lorentzians):
        center = params[i * 3]
        sigma = params[i * 3 + 1]
        amplitude = params[i * 3 + 2]
        result += lorentzian(x, center, sigma, amplitude)
    return result

# generate some sample data
xdata = np.linspace(-10, 10, 201)
ydata = lorentzian(xdata, 0, 0, 1, 1) + np.random.normal(0, 0.1, xdata.shape)

# define the model and set initial parameter values
#INITIALIZA THE FUNCTIONS 
#%%
#import the necessary packages
import automatization
from automatization.ple_statistics import PleAuto
import pandas as pd
import os
import time
import json
from importlib import reload
import numpy as np
from automatization.ple_statistics import PleAuto
from matplotlib import pyplot as plt
reload(automatization.ple_statistics)
from automatization.ple_statistics import PleAuto
from scipy.constants import speed_of_light
import pickle
from pathlib import Path

import sys

sys.setrecursionlimit(10000)

pa = PleAuto(
    ple_gui, 
    ple_optimize_logic, 
    laser_scanner_logic,
    poi_manager_logic, 
    scanning_optimize_logic,
    spectrometer,  
    
    spectrometerlogic,
    cobolt,
    ibeam_smart
)

# def ple_is_here(res, center_sigma = 3e3, amplitude = 1000, sigma_stderr_ratio = 4, amplitude_stderr_ratio=3):
#     """Check if the ple is still there."""
#     it_is = True
#     if res["center"].stderr is None or res["sigma"].stderr is None or res["sigma"].value is None:
#         return False
#     if ( res["center"].stderr > center_sigma or 
#     res["sigma"].stderr * sigma_stderr_ratio > res["sigma"].value or 
#     res["amplitude"].stderr * amplitude_stderr_ratio > res["amplitude"].value or
#     res["amplitude"].value < amplitude): 
#         #ple is gone.
#         return False

#     return it_is
def ple_is_here(rsquared_bound = 0.5):
    """Check if the ple is still there."""
    
    if ple_gui.fit_result[1].rsquared < rsquared_bound:
        print("NO, PLE", ple_gui.fit_result[1].rsquared )
        return False
    
    else:
        return True


def adjust_eta(pa, poi_name, folder_defect, results_poi, center_v):
    """Adjust the etalon voltage to get the best fit."""
    # ple_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["a"])
    ple_gui._mw.frequencyDoubleSpinBox.setValue(1000)
    
    eta_volts = [center_v, center_v + 0.2, center_v + 0.4, center_v + 0.6, center_v - 0.2, center_v - 0.4,  center_v - 0.6]
    r_squared = []
    for eta_v in eta_volts:
        laser_controller_remote.etalon_voltage = eta_v
        time.sleep(0.5)
        res = pa.do_ple_scan(lines = 1)
        time.sleep(0.5)
        r_squared.append(ple_gui.fit_result[1].rsquared)
    results_poi["eta_voltage"] = eta_volts[r_squared.index(max(r_squared))]
    laser_controller_remote.etalon_voltage = eta_volts[r_squared.index(max(r_squared))]
    res = pa.do_ple_scan(lines = 1)
    pa.save_ple(tag = "full_range_eta_adjusted",
                poi_name=poi_name, folder_name=folder_defect)

    ple_gui._mw.frequencyDoubleSpinBox.setValue(250)
    return res, results_poi
def ple_refocus(pa, opt_times = 1, 
                scan_frequency=200, 
                scan_resolution=500,
                scan_range = 5000):
    
    seqs = {str(seq): idx for idx, seq in enumerate(ple_gui._osd.settings_widget.available_opt_sequences)}
    ple_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["a"])
    ple_gui._osd.change_settings({'scan_frequency': {"a": scan_frequency},
                                    "scan_resolution": {"a":scan_resolution},
                                    "scan_range": {"a": scan_range}}) #GHz

    ple_gui._osd.accept()
    time.sleep(0.8)
    pa.optimize_ple()
    return res
def settings_confocal_refocus(sequence = "xy, z", scan_range_xy= 1e-6, scan_resolution = 25, scan_frequency = 25):
    seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs[sequence])
    scanner_gui._osd.change_settings({'scan_frequency': {"x": scan_frequency, "y": scan_frequency, "z": scan_frequency},
                                    "scan_resolution": {"x": scan_resolution, "y": scan_resolution, "z":80},
                                    "scan_range": {"x": scan_range_xy, "y": scan_range_xy, "z": 4e-6}})

    scanner_gui._osd.accept()
    time.sleep(0.5)


def settings_confocal_refocus_coarse():
    seqs = {str(seq): idx for idx, seq in enumerate(scanner_gui._osd.settings_widget.available_opt_sequences)}
    scanner_gui._osd.settings_widget.optimize_sequence_combobox.setCurrentIndex(seqs["x, y, z"])

    scanner_gui._osd.change_settings({'scan_frequency': {"x": 5, "y": 5, "z": 5},
                                    "scan_resolution": {"x": 80, "y": 80, "z":80},
                                    "scan_range": {"x": 2.5e-6, "y": 2.5e-6, "z": 4.5e-6}})
    scanner_gui._osd.accept()
    time.sleep(0.5)

def confocal_refocus(opt_times=2):
    for i in range(opt_times):
        scanning_optimize_logic.start_optimize()
        while scanning_optimize_logic.module_state()=='locked':
            time.sleep(0.5)
    time.sleep(0.5)
#find the defect:
def find_the_defect(pa, poi_name, folder_defect):
    ple_gui._mw.frequencyDoubleSpinBox.setValue(1000)
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    pa.set_resonant_power(power = 300)
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(2)
    time.sleep(1)
    settings_confocal_refocus_coarse()
    confocal_refocus(opt_times=2)

    
    switchlogic.set_state("ScanningMode", 'NI')
    time.sleep(0.5)
    #Check how the PLE look like
    res = pa.do_ple_scan(lines = 1)

    #configure slow scanning for the wavemeter scanning optimizations
    for kk in range(3):
        if not ple_is_here():
            switchlogic.set_state("ScanningMode", 'Wavemeter')
            time.sleep(0.5)
            settings_confocal_refocus_coarse()
            confocal_refocus(opt_times=1)
            #Check how the PLE look like
            time.sleep(0.5)
            switchlogic.set_state("ScanningMode", 'NI')
            res = pa.do_ple_scan(lines = 1)
            time.sleep(0.5)
            pa.save_ple(tag = "full_range_iter_{kk}",
                    poi_name=poi_name, folder_name=folder_defect)
            time.sleep(0.5)
        else:
            break

    pa.save_ple(tag = "full_range",
            poi_name=poi_name, 
            folder_name=folder_defect)
    time.sleep(0.2)
    ple_gui._mw.frequencyDoubleSpinBox.setValue(250)
    return res

def take_spectrum(pa, poi_name, folder_defect):
    # take spectrum to estimate SOC
    
    
    ibeam_smart.setPower(60000)
    ibeam_smart.enable()

    pa.take_spectrum()

    time.sleep(0.2)
    spectrometer._mw.data_widget.fit_widget.sigDoFit.emit("DoubleLorentzian")
    time.sleep(0.2)
    pa.save_spectrum(name_tag=f"{poi_name}_blueNgreen", folder_path=folder_defect)
    # spectrometer.fit_results.params["center_1"].value
    
    if spectrometer.fit_results is not None:
        params = spectrometer.fit_results.params
        soc_ghz = float(speed_of_light / params["center_2"].value - speed_of_light / params["center_1"].value)
        center_1_ghz = float(params["center_1"].value)
        center_2_ghz = float(params["center_2"].value)

    return soc_ghz, center_1_ghz, center_2_ghz
# Perform the saturation measurement

def fine_optimize(pa, poi_name, folder_defect, results_poi):
    res = pa.do_ple_scan(lines = 1)
    pa.go_to_ple_target(res["center"].value)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    settings_confocal_refocus_fine()
    confocal_refocus(opt_times=1)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    confocal_refocus(opt_times=1)
    res = pa.do_ple_scan(lines = 1)
    pa.go_to_ple_target(res["center"].value)
    results_poi["center"] = res["center"].value
    
    results_poi["center_λ"] = high_finesse_wavemeter_remote.get_current_wavelength()
    pa.save_ple(tag = "full_range_optimized",
            poi_name=poi_name, 
            folder_name=folder_defect)
    return res, results_poi

def run_saturation_measurement(pa, res, poi_name, folder_defect, results_poi, large_sigma = 4000, small_sigma = 300, power_steps = None, low_power_steps = None, repumped_power = 100, no_repump_power = 150):
    ple_gui._mw.frequencyDoubleSpinBox.setValue(500)

    os.makedirs(saturation_folder := os.path.join(folder_defect, "saturation"), exist_ok=True)
    results_poi["saturation"] = {}
    idx_no_ple = None
    res_old = res
    if power_steps is None:
        power_steps = 3 * np.logspace(1.5, 2, 10, endpoint=True).astype(int)[::-1]
    if low_power_steps is None:
        low_power_steps = np.array([85, 78, 70, 65, 60])
    power_steps = np.append(power_steps,low_power_steps)
    zero_range = (
                res["center"].value - large_sigma,
                res["center"].value + large_sigma
            )
    res = pa.do_ple_scan(lines = 1, in_range=zero_range)
    for idx, power in enumerate(power_steps):
        os.makedirs(power_folder := os.path.join(saturation_folder, f"{power}"), exist_ok=True)
        
        if power > repumped_power: 
            cobolt.enable_modulated()
            pa.set_resonant_power(power = power)
            time.sleep(1)
            #align twice
            # for i in range(2)
            
            fine_range = (
                res["center"].value - small_sigma,
                res["center"].value + small_sigma
            )
            if res["center"].value is None or res["sigma"].value is None or not ple_is_here(rsquared_bound=0.2):
                res = pa.do_ple_scan(lines = 1, in_range=zero_range)
                
                if not ple_is_here(rsquared_bound=0.2):
                    continue
            res = pa.do_ple_scan(lines = 1, in_range=fine_range)
            # if abs(res_old["center"].value - res["center"].value) > res_old["sigma"].value*2:
            #     res = res_old
            # else:s
            #     res_old = res
            pa.save_ple(tag = f"{power}",
                poi_name=poi_name, folder_name=power_folder)

            results_poi.update({"saturation": 
                            {f"{power}_repump":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res["sigma"].value,
                                "sigma_stderr": res["sigma"].stderr,
                                "center": res["center"].value
                                }
                                }})
        if (power < no_repump_power):
            os.makedirs(power_folder_norepump := os.path.join(power_folder, f"no_repump"), exist_ok=True)
            # check with initioalization
            cobolt.disable_modulated()
            pa.one_pulse_repump("violet")
            for i in range(3):
                pa.one_pulse_repump("violet")
                res_ = pa.do_ple_scan(lines = 1, in_range=fine_range)
                if not ple_is_here(rsquared_bound=0.4):
                    continue
            if not ple_is_here(rsquared_bound=0.4):
                    continue
            res_ = pa.do_ple_scan(lines = 5, in_range=fine_range)
            print("Saving no repump")
            pa.save_ple(tag = f"{power}_norepump",
                poi_name=poi_name, folder_name=power_folder_norepump)
            results_poi.update({"saturation": 
                            {f"{power}_norepump":
                                {"scan_data": ple_data_logic.last_saved_files_paths,
                                "sigma": res_["sigma"].value,
                                "sigma_stderr": res_["sigma"].stderr,
                                "center": res_["center"].value
                                }
                                }})
        if power <= 40:
            break

            #save_plots
    return results_poi

def check_factor_sigma():
    # constraints
    scan_range_constr = laser_scanner_logic.scan_ranges["a"] 
    pa.go_to_ple_target(scan_range_constr[0])
    time.sleep(5)
    w1 = high_finesse_wavemeter_remote.get_current_wavelength()
    
    pa.go_to_ple_target(scan_range_constr[1])
    time.sleep(5)
    w2 = high_finesse_wavemeter_remote.get_current_wavelength()

    factor_sigma = (w2 - w1) * 1000000 / (scan_range_constr[1] - scan_range_constr[0]) #GHz 
    return factor_sigma

def get_optical_power(wavelength = 619):
    powermeter._set_wavelength(value=619)
    powermeter.set_activity_state(channel="Power", active=True)
    power = powermeter.get_process_value(channel="Power")
    return power * 1e9 # in nW

# constraints
scan_range_constr = laser_scanner_logic.scanner_constraints.axes["a"].value_range
scanning_optimize_logic._backwards_line_resolution = 20



import os

SCAN_RANGE = 1000
LARGE_SCAN_RANGE = 5000
c = 299792458

def setup_wavemeter_scan(repump_power):
    high_finesse_wavemeter_remote._wavemeter.set_regulation_mode(mode=True)
    switchlogic.set_state("ScanningMode", 'Wavemeter')
    cobolt.set_laser_modulated_power(repump_power)
    cobolt.enable_modulated()
    ibeam_smart.disable()
    pa.set_resonant_power(300)

def disable_wavemeter_scan():
    high_finesse_wavemeter_remote._wavemeter.set_regulation_mode(mode=False)
    switchlogic.set_state("ScanningMode", 'NI')


def append_to_file(file_path, content):
    with open(file_path, "a") as f:
        f.write(content)

def find_ple_center():
    
    for i in range(3):
        res = pa.do_ple_scan(lines=1)
        if ple_is_here(rsquared_bound=0.15):
            pa.go_to_ple_target(res["center"].value)
            ple_refocus(pa, scan_range=4000, scan_frequency=500)
            res_center = laser_scanner_logic.scanner_target["a"]
            res = pa.do_ple_scan(lines=1, in_range=(res_center - LARGE_SCAN_RANGE, res_center + LARGE_SCAN_RANGE))
            if ple_is_here(rsquared_bound=0.6):
                break
        else:
            search_with_wavemeter_scan(opt_times=1)
    pa.go_to_ple_target(res["center"].value)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    res_center = laser_scanner_logic.scanner_target["a"]
    return res_center

def search_with_wavemeter_scan(repump_power=0.2, opt_times=2):
    setup_wavemeter_scan(repump_power = repump_power)
    settings_confocal_refocus(sequence="x, y, z", scan_resolution=80, scan_frequency=5, scan_range_xy=3e-6)
    confocal_refocus(opt_times=opt_times)
    disable_wavemeter_scan()


def save_ple_data_and_optimize(poi_name, folder_defect, res_center):
    pa.save_ple(tag=f"ple_scan_{i}", poi_name=poi_name, folder_name=folder_defect)
    pa.go_to_ple_target(res_center)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)

    wavelength = 1e-3 * c / high_finesse_wavemeter_remote.get_current_wavelength()
    append_to_file(os.path.join(folder_defect, "info.txt"), f"lambda = {wavelength} nm\n")

    #optimize in resonance
    cobolt.set_laser_modulated_power(5) #for optimization
    ple_refocus(pa, scan_range=LARGE_SCAN_RANGE, scan_frequency=500)
    settings_confocal_refocus(sequence="xy, z", scan_resolution=20, scan_frequency=100, scan_range_xy=1e-6)
    confocal_refocus(opt_times=1)
    ple_refocus(pa, scan_range=LARGE_SCAN_RANGE, scan_frequency=500)    
    settings_confocal_refocus(sequence="xy, z", scan_resolution=20, scan_frequency=100, scan_range_xy=1e-6)
    confocal_refocus(opt_times=1)
    cobolt.set_laser_modulated_power(repump_power_405)
    res = pa.do_ple_scan(lines = 1, in_range=(res_center - LARGE_SCAN_RANGE, res_center + LARGE_SCAN_RANGE))
    pa.go_to_ple_target(res["center"].value)
    ple_refocus(pa, scan_range=4000, scan_frequency=500)
    res_center = laser_scanner_logic.scanner_target["a"]

def perform_ple_measurements(poi_name, folder_defect, res_center):
    for power in [300, 250, 200, 180, 160]:
        power_folder = create_directory(os.path.join(folder_defect, f"{power}"))
        pa.set_resonant_power(power)
        cobolt.set_laser_modulated_power(repump_power_405)
        cobolt.enable_modulated()
        ibeam_smart.disable()

        pa.go_to_ple_target(res_center)
        ple_refocus(pa, scan_range=1000, scan_frequency=500)
        res_center_ = laser_scanner_logic.scanner_target["a"]

        res = pa.do_ple_scan(lines=1, in_range=(res_center_ - LARGE_SCAN_RANGE, res_center_ + LARGE_SCAN_RANGE))
        if not ple_is_here(rsquared_bound=0.2):
            res = pa.do_ple_scan(lines=1)
            if ple_is_here(rsquared_bound=0.4):
                pa.go_to_ple_target(res["center"].value)
                ple_refocus(pa, scan_range=3000, scan_frequency=500)
                res_center = laser_scanner_logic.scanner_target["a"]
            elif power > 200:
                break
        else:
            res_center = res_center_
        ple_params = f"\npower = {power}\n center = {res['center'].value}\n sigma = {res['sigma'].value}\n sigma_err = {res['sigma'].stderr}\nscan_range = {LARGE_SCAN_RANGE}\n violet repump\n\n"
        append_to_file(os.path.join(power_folder, f"ple_scan_power_{power}.txt"), ple_params)

        pa.save_ple(tag=f"power_{power}", poi_name=poi_name, folder_name=power_folder)

        res_ = pa.do_ple_scan(lines=1, in_range=(res_center - SCAN_RANGE, res_center + SCAN_RANGE))
        ple_params = f"\npower = {power}\n center = {res_['center'].value}\n sigma = {res_['sigma'].value}\n sigma_err = {res_['sigma'].stderr}\nscan_range = {SCAN_RANGE}\n violet repump\n\n"
        append_to_file(os.path.join(power_folder, f"ple_scan_power_{power}.txt"), ple_params)

        pa.save_ple(tag=f"power_{power}", poi_name=poi_name, folder_name=power_folder)


        if power <= 200:
            ple_gui._mw.frequencyDoubleSpinBox.setValue(350)
            ple_gui._mw.resolutionDoubleSpinBox.setValue(2000)

            # Prescan repump
            ple_gui._mw.Pulsed_widget.prescan_checkBox.setChecked(True)
            res = pa.do_ple_scan(lines=10, in_range=(res_center - SCAN_RANGE, res_center + SCAN_RANGE))
            pa.save_ple(tag=f"power_{power}_prescan_repump", poi_name=poi_name, folder_name=power_folder)

           
            ple_params = f"PRE_SCAN_REPUMP:\npower = {power}\n center = {res['center'].value}\n sigma = {res['sigma'].value}\n sigma_err = {res['sigma'].stderr}\nscan_range = {SCAN_RANGE}\n violet repump\n\n"
            with open(os.path.join(power_folder, f"ple_scan_power_{power}.txt"), "a") as f:
                f.write(ple_params)
            ple_rescan_here = ple_is_here(rsquared_bound=0.2)
            #no repump
            ple_gui._mw.Pulsed_widget.prescan_checkBox.setChecked(False)
            cobolt.disable_modulated()
            res = pa.do_ple_scan(lines = 10, in_range=(
                res_center - SCAN_RANGE, res_center + SCAN_RANGE))
            pa.save_ple(tag = f"power_{power}_NO_repump", 
                        poi_name=poi_name, folder_name=power_folder)
            ple_params = f"NO_REPUMP:\npower = {power}\n center = {res['center'].value}\n sigma = {res['sigma'].value}\n sigma_err = {res['sigma'].stderr}\nscan_range = {SCAN_RANGE}\n violet repump\n\n"
            with open(os.path.join(power_folder, f"ple_scan_power_{power}.txt"), "a") as f:
                f.write(ple_params)
            ple_norepump_here = ple_is_here(rsquared_bound=0.2)
            if not ple_norepump_here and not ple_rescan_here:
                break
            
    ple_gui._mw.Pulsed_widget.prescan_checkBox.setChecked(False)
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(repump_power_405)
def run_450nm(on_off):
    seq = pulsestreamer.pulse_streamer.createSequence()
    # pulse_patt = [(100, 0), (200, 1), (80, 0), (300, 1), (60, 0)]

    seq.setDigital(2, [(100, int(on_off))])
    seq.setDigital(6, [(100, 1)])
    seq.setDigital(7, [(100, 0)])
    # Stream the continuous signal
    period = 1000  # Period in ns (1 us)
    n_runs = 0  # Number of runs (0 for continuous streaming)
    pulsestreamer.pulse_streamer.stream(seq)

def take_and_save_spectrum(poi_name, folder_defect):
    os.makedirs(spectrum_folder := os.path.join(folder_defect, f"spectrum"), exist_ok=True)
    
    soc_ghz, center_1_ghz, center_2_ghz = take_spectrum(pa, poi_name, spectrum_folder)

    line = f"\nsoc_ghz = {soc_ghz} GHz, \ncenter_1_ghz = {center_1_ghz} GHz, \ncenter_2_ghz = {center_2_ghz} GHz\n"
    with open(os.path.join(folder_defect, "info.txt"), "a") as f:
        f.write(line)


In [108]:
#Function to run repump lasers
def run_450nm(on_off, on_time = 1e3, off_time=0):
    seq = pulsestreamer.pulse_streamer.createSequence()
    # pulse_patt = [(100, 0), (200, 1), (80, 0), (300, 1), (60, 0)]
    if on_off:
        seq.setDigital(2, [(on_time, 1), (off_time, 0)])
    else:
        seq.setDigital(2, [(on_time, 0), (off_time, 0)])
    pulsestreamer.pulse_streamer.stream(seq)

def run_405nm(on_off, on_time = 1e3, off_time=1e6, power=0.1):
    cobolt.enable_modulated()
    cobolt.set_laser_modulated_power(power)
    seq = pulsestreamer.pulse_streamer.createSequence()
    # pulse_patt = [(100, 0), (200, 1), (80, 0), (300, 1), (60, 0)]
    if on_off:
        seq.setDigital(7, [(on_time, 1), (off_time, 0)])
    else:
        seq.setDigital(7, [(on_time, 0), (off_time, 0)])
    pulsestreamer.pulse_streamer.stream(seq)
def run_all_repumps():
    seq = pulsestreamer.pulse_streamer.createSequence()
    seq.setDigital(2, [(100, 1), (0, 0)])
    seq.setDigital(7, [(100, 1), (0, 0)])
    pulsestreamer.pulse_streamer.stream(seq)

#enable the modulation of 405 nm repump (controlled by pulse streamer)
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(1)

In [5]:
# factor_sigma = check_factor_sigma()
# factor_sigma

In [3]:
folders = {"1200_40keV_5e9": 
                {"ref": r"Z:\Vlad\SnV\TinGeneration\158-#1D\Area40x40",
                "ref-etched": r"Z:\Vlad\SnV\TinGeneration\158-#1C\Area40x40",
                "pplus-etched": r"Z:\Vlad\SnV\TinGeneration\158-#1A\Area40x40"},
            "1200_40keV_2e10":           
                {"ref": r"Z:\Vlad\SnV\TinGeneration\158-#2D\Area40x40",
                "ref-etched": r"Z:\Vlad\SnV\TinGeneration\158-#2C\Area40x40",
                "pplus-etched": r"Z:\Vlad\SnV\TinGeneration\158-#2A\Area40x40",},
           "1200_170keV_5e9":           
                {"ref": r"Z:\Vlad\SnV\TinGeneration\158-#3D\Area40x40",
                "ref-etched": r"Z:\Vlad\SnV\TinGeneration\158-#3C\Area40x40",
                "pplus-etched": r"Z:\Vlad\SnV\TinGeneration\158-#3A\Area40x40",},
           }

In [6]:
folder = folders["1200_40keV_5e9"]["ref"]

high_finesse_wavemeter_remote.start_acquisition()

0

In [115]:
ibeam_smart.enable()

### PLE search

In [325]:
#405 nm cw repump The violet is more 
run_all_repumps()
# run_405nm(100)
# run_450nm(True)
cobolt.enable_modulated()
cobolt.set_laser_modulated_power(1)
ibeam_smart.disable()

#set max power to serach for PLE
power = 300
powercontroller_logic.sig_set_power.emit(power, 3, False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

ple_gui._save_folderpath = folder_saturation
ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"repump_405n450nm_power_{power}")

In [ ]:
laser_

In [100]:
run_450nm(1, on_time=1e3, off_time=1e6)

In [179]:
run_405nm(100, power=5)

In [332]:
laser_controller_remote.etalon_voltage = -5.8

### PLE is found

In [326]:
# initialize folders
poi_manager_gui._mw.new_poi_Action.triggered.emit()
time.sleep(0.2)
poi_name = poi_manager_logic.active_poi#"def1"

os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)
os.makedirs(folder_450_repumped := os.path.join(folder_defect, "450_repumped"),  exist_ok=True)
os.makedirs(folder_no_rep := os.path.join(folder_defect, "no_repump"),  exist_ok=True)
os.makedirs(folder_prerepumped := os.path.join(folder_defect, "prescan_repumped"),  exist_ok=True)
os.makedirs(folder_405_repumped := os.path.join(folder_defect, "405_repumped"),  exist_ok=True)
os.makedirs(folder_spectrum := os.path.join(folder_defect, "spectrum"),  exist_ok=True)
os.makedirs(folder_saturation := os.path.join(folder_defect, "saturation"),  exist_ok=True)
spectrometer._save_folderpath = folder_spectrum
spectrometer._mw.control_widget.save_path_widget.currPathLabel.setText(folder_spectrum)
spectrometer._mw.control_widget.save_path_widget.newPathCheckBox.setChecked(False)



In [327]:
ple_gui._save_folderpath = folder_saturation
ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"repump_405n450nm_power_{power}")

In [333]:
wavelength = 1e-3 * c / high_finesse_wavemeter_remote.get_current_wavelength()
append_to_file(os.path.join(folder_defect, "info.txt"), f"lambda = {wavelength} nm\n")

In [335]:
#set max power to serach for PLE
power = 240

powercontroller_logic.sig_set_power.emit(power, 3, False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

In [342]:
polarization_angle =30
motordriver_pi3.moveToAbsolutePosition(motor=0, pos=polarization_angle)

In [343]:
#Optimize polarization
append_to_file(os.path.join(folder_defect, "info.txt"), f"optimal polarization = {polarization_angle} deg\n")

In [256]:
#Go down with power:
power = 90
powercontroller_logic.sig_set_power.emit(power, 3, False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

In [347]:
#set max power to serach for PLE
power = 190

powercontroller_logic.sig_set_power.emit(power, 3, False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

In [344]:
#450 nm cw repump
run_450nm(1, on_time=1e3, off_time=1e6)

ple_gui._save_folderpath = folder_450_repumped
ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"repump_450nm_power_{power}")

In [145]:
#Go down with power:
power = 100

powercontroller_logic.sig_set_power.emit(power, 3, False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

In [289]:
run_405nm(1, on_time=1e3, off_time=1e6)

ple_gui._save_folderpath = folder_405_repumped
ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)

ple_gui.save_path_widget.saveTagLineEdit.setText(f"repump_405nm-1e3on-1e6off_power_{power}")

In [119]:
# Minimal power PLE

In [274]:
#Go down with power:
power = 100
powercontroller_logic.sig_set_power.emit(power, 3, False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

In [141]:
#NO repump
run_450nm(1)
time.sleep(2)
run_450nm(0)

In [351]:
#NO repump
run_405nm(1)
time.sleep(0.1)
run_405nm(0)

In [348]:
#NO repump
run_450nm(1)
time.sleep(1)
run_450nm(0)

ple_gui._save_folderpath = folder_no_rep
ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

In [219]:
#Now repumped PLE
run_450nm(1, on_time=1e3, off_time=1e8)

ple_gui._save_folderpath = folder_no_rep
ple_gui.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
ple_gui.save_path_widget.newPathCheckBox.setChecked(False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"rare_450nmrepump_1e3_1e8_power_{power}")

In [169]:
power = 120
powercontroller_logic.sig_set_power.emit(power, 3, False)
ple_gui.save_path_widget.saveTagLineEdit.setText(f"power_{power}")

In [277]:
factor_sigma = check_factor_sigma()
line = f"\nfactor_sigma = {factor_sigma}\n"
with open(os.path.join(folder_defect, "info.txt"), "a") as f:
    f.write(line)
print("Factor sigma", factor_sigma)

Factor sigma 1.0245034165998619


### Take spectrum

In [360]:
folder = folders["1200_40keV_5e9"]["ref"]
os.makedirs(folder_spectra := os.path.join(folder, "spectra"),  exist_ok=True)

In [ ]:
switch_combiner_interfuse.set_state("ResonantShutter", "On")
ibeam_smart.setPower(60000)
ibeam_smart.enable()
run_450nm(0)

In [ ]:
for poi_name in poi_manager_logic.poi_names:

    settings_confocal_refocus(sequence="xy, z", scan_resolution=30, scan_frequency=25, scan_range_xy=2.5e-6)
    pa.go_to_poi(poi_name, ref_poi=None, opt_times=2)
    os.makedirs(folder_defect := os.path.join(folder, poi_name),  exist_ok=True)
    spectrometer._save_folderpath = folder_defect

    spectrometer._mw.control_widget.save_path_widget.currPathLabel.setText(ple_gui._save_folderpath)
    spectrometer._mw.control_widget.save_path_widget.newPathCheckBox.setChecked(False)

    spectrometer._mw.control_widget.background_button.clicked.emit()

    while spectrometerlogic._acquisition_running:
            time.sleep(1)
            
    time.sleep(0.5)

    take_and_save_spectrum(poi_name, folder_defect)




In [352]:
ibeam_smart.disable()
run_450nm(1)
switch_combiner_interfuse.set_state("ResonantShutter", "Off")